## IMPORT LIBRARIES

In [1]:
# Import necessary libraries
from pyDOE2 import fracfact
from pyDOE2 import ff2n
import numpy as np
import pandas as pd
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
from statsmodels.stats.libqsturng import psturng
from statsmodels.stats.multitest import multipletests
from scipy.stats import t
import itertools
from itertools import combinations
from pyDOE2 import pbdesign


## CREATE FACTORIAL DESIGN PLAN

In [2]:
# Function to generate a DataFrame with factor names
def generate_factors_df(factors):
    """
    Generate a DataFrame with factor names.

    Args:
    - factors (list): List of factor names.

    Returns:
    - df (DataFrame): DataFrame representing factor names.
    """
    data = {'Factor': factors}
    df = pd.DataFrame(data)
    return df

# Function to generate a Plackett-Burman design matrix based on given factors
def plackett_burman_design(factors_df):
    """
    Generates a Plackett-Burman design matrix based on the given factors.

    Args:
    - factors_df (DataFrame): DataFrame containing the factor names.

    Returns:
    - df (DataFrame): DataFrame containing the design matrix.
    """
    factors = factors_df['Factor'].tolist()
    
    # Generate Plackett-Burman design
    design = pbdesign(len(factors))

    design = -design

    # Convert design matrix back to original factors
    df = pd.DataFrame(design, columns=factors)
    
    return df

# Define the custom mapping function
def custom_mapping(value):
    """
    Custom mapping function to transform values according to specific rules.

    Args:
    - value (int): Value representing factor level.

    Returns:
    - mapped_value (int): Transformed value.
    """
    # Map values according to specific rules
    if value == 1:
        return 0 if 2 in design_matrix.values.flatten() else 1
    elif value == 2:
        return 1
    elif value == 0:
        return -1
    else:
        return value

In [3]:
# Prompt user for factors
while True:
    try:
        factors = input("Enter the factors separated by spaces: ").split()
        factors_df = generate_factors_df(factors)
        break
    except ValueError:
        print("Invalid input. Please enter factors separated by spaces.")

# Generate Plackett-Burman design based on factors
design_matrix = plackett_burman_design(factors_df)

# Apply custom mapping to each value in the design matrix
design_matrix = design_matrix.applymap(custom_mapping)

# Prompt the user to enter the number of replicates
while True:
    try:
        num_replicates = int(input("Enter the number of replicates: "))
        if num_replicates <= 0:
            raise ValueError("Number of replicates must be a positive integer.")
        break
    except ValueError:
        print("Invalid input. Please enter a positive integer for the number of replicates.")

# Concatenate the design matrix DataFrame with itself
duplicated_design = pd.concat([design_matrix] * num_replicates, ignore_index=True)

# Merge the two DataFrames
merged_df = pd.concat([factors_df, pd.DataFrame(columns=['']), duplicated_design], axis=1)

# Add a blank column with heading 'Results' after the last factor
merged_df.insert(len(merged_df.columns), 'Results', '')

# Prompt the user to input the Excel file name
excel_file_name = input("Enter the Excel file name: ")

# Append '.xlsx' extension if not provided
if not excel_file_name.endswith('.xlsx'):
    excel_file_name += '.xlsx'

# Write the merged DataFrame to an Excel file
try:
    merged_df.to_excel(excel_file_name, index=False)
    print("Excel file saved successfully.")
except Exception as e:
    print("Error occurred while saving the Excel file:", e)

Excel file saved successfully.
